In [1]:
# -*- coding: utf-8 -*-
"""
Created on 12-10-2019

@author: Xin Huang
"""

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop


from sklearn.model_selection import train_test_split
import numpy as np

# data = np.load('Sat_data_small.npz')
data = np.load('train10.npz')

# passive = 1

#load common data
latlon = data['latlon']
iff = data['iff']

X_v = data['viirs']
Y_v = data['label']
print ('X_v shape:');
print (X_v.shape);

X_c = data['calipso']
Y_c = data['label']
print ('X_c shape:');
print (X_c.shape);

inds,vals = np.where(Y_v>0)

# process common data
Latlon = latlon[inds]
Iff = iff[inds]

Y_v = Y_v[inds]
X_v = X_v[inds]
print ('X_v')
print (X_v)

# inds_c,vals_c = np.where(Y_c>0)

Y_c = Y_c[inds]
X_c = X_c[inds]
print ('X_c')
print (X_c)



# 0 =< SZA <= 83
print('original X_v: ', X_v.shape)
rows = np.where((X_v[:,0] >= 0) & (X_v[:,0] <= 83) & (X_v[:,15] > 100) & (X_v[:,15] < 400) & (X_v[:,16] > 100) & (X_v[:,16] < 400) & (X_v[:,17] > 100) & (X_v[:,17] < 400) & (X_v[:,18] > 100) & (X_v[:,18] < 400) & (X_v[:,19] > 100) & (X_v[:,19] < 400) & (X_v[:,10] > 0))
print("rows:", rows)
print("rows.shape:", len(rows))

Latlon = Latlon[rows]
Iff = Iff[rows]

Y_v = Y_v[rows]
X_v = X_v[rows]

Y_c = Y_c[rows]
X_c = X_c[rows]

X_c = np.nan_to_num(X_c)
X_v = np.nan_to_num(X_v)

print('after SZA X_v: ', X_v.shape)
print('after SZA X_c: ', X_c.shape)

# pca = decomposition.PCA(n_components=20)
# pca.fit(X_s)
# X_s = pca.transform(X_s)
# print (X_s.shape)

#concanate common data
# X_v = np.concatenate((X_v, Latlon, Iff), axis=1)
X_c = np.concatenate((X_c, Latlon, Iff), axis=1)

print (X_v.shape)
print (X_c.shape)


Using TensorFlow backend.
/Users/xin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/xin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/xin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/xin/anaconda3/lib/python3.7/site-packages/tensorflow/py

X_v shape:
(2099451, 20)
X_c shape:
(2099451, 25)
X_v
[[ 97.43999481 -55.29999924   2.52999997 ... 246.61909485 246.91960144
  246.64631653]
 [ 97.43999481 -55.2899971    2.51999998 ... 247.22109985 247.5663147
  247.28843689]
 [ 97.40999603 -55.22999954   2.5        ... 248.27958679 248.7721405
  248.57876587]
 ...
 [ 94.68000031 -62.64999771   2.12999988 ... 228.96557617 228.62413025
  227.59910583]
 [ 94.68000031 -62.62999725   2.12999988 ... 229.51361084 229.17892456
  227.92941284]
 [ 94.66999817 -62.6099968    2.12999988 ... 230.20637512 230.03952026
  228.89453125]]
X_c
[[ 1.000e+00  2.000e+00  0.000e+00 ...        nan -9.999e+03        nan]
 [ 1.000e+00  2.000e+00  0.000e+00 ...        nan -9.999e+03        nan]
 [ 1.000e+00  2.000e+00  0.000e+00 ...        nan -9.999e+03        nan]
 ...
 [ 1.000e+00  1.000e+00  0.000e+00 ...        nan -9.999e+03        nan]
 [ 1.000e+00  1.000e+00  0.000e+00 ...        nan -9.999e+03        nan]
 [ 1.000e+00  1.000e+00  0.000e+00 ...        

In [2]:
print(X_c[:,0])
print(X_c[:,2])
print(X_c[:,1].shape)

[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 0. 0. 0.]
(704800,)


In [4]:
# combine data and split latter to define ground truth for MLR
from sklearn.linear_model import LinearRegression
n1=20
n2=25
X=np.concatenate((X_v, X_c), axis=1)
Y=Y_v

Y=Y-1
print (X.shape)
print (Y_v)
x_train, x_valid, y_train, y_valid = train_test_split(X, Y,
                                                    test_size=0.3,
                                                    random_state=0,
                                                    stratify=Y)

# x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp,
#                                                     test_size=0.5,
#                                                     random_state=0,
#                                                     stratify=y_temp)


# # feature scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_X.fit(x_train)
x_train=sc_X.transform(x_train)
x_valid=sc_X.transform(x_valid)
# x_test=sc_X.transform(x_test)

# feature scaling
# from sklearn.preprocessing import StandardScaler
# sc_X_v = StandardScaler()
# sc_X_c = StandardScaler()


x_train_v = x_train[:, 0:20]
x_train_c = x_train[:, 20:45]
x_train_comm = x_train[:, 45:51]
# sc_X_v.fit(x_train_v)
# sc_X_c.fit(x_train_c)
print(x_train_v.shape)
print(x_train_c.shape)
print(x_train_comm.shape)
print(x_train_v[1:10, ])
print(x_train_c[1:10, ])

x_valid_v = x_valid[:, 0:20]
x_valid_c = x_valid[:, 20:45]
x_valid_comm = x_valid[:, 45:51]
# x_valid_v = sc_X_v.transform(x_valid_v)
# x_valid_c = sc_X_c.transform(x_valid_c)
print(x_valid_v.shape)
print(x_valid_c.shape)
print(x_valid_comm.shape)
print(x_valid_v[1:10, ])
print(x_valid_c[1:10, ])

# x_test_v = x_test[:, 0:20]
# x_test_c = x_test[:, 20:45]
# x_test_v = sc_X_v.transform(x_test_v)
# x_test_c = sc_X_c.transform(x_test_c)
# print(x_test_v.shape)
# print(x_test_c.shape)
# print(x_test_v[1:10, ])
# print(x_test_c[1:10, ])

(704800, 51)
[[2]
 [2]
 [2]
 ...
 [3]
 [3]
 [3]]
(493360, 20)
(493360, 25)
(493360, 6)
[[-7.33811578e-02 -1.15862914e+00  8.21589628e-01 -2.13746582e-01
   1.99623797e+00  2.01215955e+00  1.98477600e+00  1.88350589e+00
   1.69663707e+00  6.92207008e-01  1.26696591e+00  6.06449143e-02
  -3.94587737e-01 -7.82200217e-01 -6.94167704e-01 -8.56558481e-01
  -4.81644112e-01 -1.49800418e-01 -1.26641380e-01 -9.40186576e-02]
 [ 6.25914276e-02  2.04895210e-01 -1.16085896e+00 -2.88055129e-01
  -1.09706561e+00 -1.06784619e+00 -1.02367801e+00 -9.02692722e-01
  -7.93362880e-01  5.22677969e-01 -3.54132287e-01  1.10853550e-01
  -4.09015819e-01  5.40459857e-01  2.59291590e-01  1.40943711e+00
   1.54027862e+00  1.59139303e+00  1.65528737e+00  1.70020683e+00]
 [ 1.35700696e+00  1.18958808e-01  2.62199887e-01 -3.74379567e-01
  -1.21855056e+00 -1.16112649e+00 -1.11789304e+00 -1.08221613e+00
  -1.07099576e+00 -1.87262899e+00 -1.12241390e+00 -1.09622570e+00
  -4.19836883e-01 -9.34002519e-01 -9.99211466e-01 -4.

In [5]:
# #use MLP to impute the virrs data to Calipso
# from sklearn.neural_network import MLPRegressor
# # hidden = 100
# mlpReg = MLPRegressor(activation = 'relu',hidden_layer_sizes = (64, 128, 128,64,),
#                                max_iter=1000,
#                                warm_start=True,solver='adam')
# mlpReg.fit(x_train_v, x_train_c)
# x_train_pt = mlpReg.predict(x_train_v)
# x_test_pt = mlpReg.predict(x_test_v)
# x_valid_pt = mlpReg.predict(x_valid_v)

# print(x_train_pt.shape)
# print(x_test_pt.shape)
# print(x_valid_pt.shape)

In [6]:
# define the sequential model with keras
model_reg = Sequential()
model_reg.add(Dense(256,input_shape=(x_train_v.shape[1],)))
model_reg.add(Dense(256, activation= "relu"))
model_reg.add(Dense(256, activation= "relu"))
model_reg.add(Dense(128, activation= "relu"))
model_reg.add(Dense(64, activation= "relu"))
model_reg.add(Dense(x_train_c.shape[1], activation= "linear"))
model_reg.summary()
model_reg.compile(loss= "mse" , optimizer='adam', metrics=['mse','mae'])
model_reg.fit(x_train_v, x_train_c, epochs=100, batch_size=128, verbose=1)

x_train_pt = model_reg.predict(x_train_v)
# x_test_pt = model_reg.predict(x_test_v)
x_valid_pt = model_reg.predict(x_valid_v)
print(x_train_pt.shape)
# print(x_test_pt.shape)
print(x_valid_pt.shape)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               5376      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_6 (Dense)              (None, 25)                1625      
Total params: 179,737
Trainable params: 179,737
Non-trainable params: 

493360/493360 [==============================] - 13s 26us/step - loss: 0.4701 - mean_squared_error: 0.4701 - mean_absolute_error: 0.2993
Epoch 45/150
493360/493360 [==============================] - 14s 27us/step - loss: 0.4936 - mean_squared_error: 0.4936 - mean_absolute_error: 0.2996
Epoch 46/150
493360/493360 [==============================] - 13s 26us/step - loss: 0.4932 - mean_squared_error: 0.4932 - mean_absolute_error: 0.2987
Epoch 47/150
493360/493360 [==============================] - 14s 28us/step - loss: 0.4864 - mean_squared_error: 0.4864 - mean_absolute_error: 0.2966
Epoch 48/150
493360/493360 [==============================] - 15s 30us/step - loss: 0.4841 - mean_squared_error: 0.4841 - mean_absolute_error: 0.2974
Epoch 49/150
493360/493360 [==============================] - 14s 28us/step - loss: 0.4951 - mean_squared_error: 0.4951 - mean_absolute_error: 0.2991
Epoch 50/150
493360/493360 [==============================] - 16s 32us/step - loss: 0.4792 - mean_squared_error: 

493360/493360 [==============================] - 13s 26us/step - loss: 0.4693 - mean_squared_error: 0.4693 - mean_absolute_error: 0.2925
Epoch 98/150
493360/493360 [==============================] - 12s 25us/step - loss: 0.4662 - mean_squared_error: 0.4662 - mean_absolute_error: 0.2882
Epoch 99/150
493360/493360 [==============================] - 12s 25us/step - loss: 0.4638 - mean_squared_error: 0.4638 - mean_absolute_error: 0.2904
Epoch 100/150
493360/493360 [==============================] - 13s 25us/step - loss: 0.4677 - mean_squared_error: 0.4677 - mean_absolute_error: 0.2914
Epoch 101/150
493360/493360 [==============================] - 12s 25us/step - loss: 0.4638 - mean_squared_error: 0.4638 - mean_absolute_error: 0.28860s - loss: 0.4668 - mean_squared_error: 0.4668 - mean_ab
Epoch 102/150
493360/493360 [==============================] - 12s 25us/step - loss: 0.4750 - mean_squared_error: 0.4750 - mean_absolute_error: 0.2963
Epoch 103/150
493360/493360 [=========================

493360/493360 [==============================] - 13s 27us/step - loss: 0.4361 - mean_squared_error: 0.4361 - mean_absolute_error: 0.2860
Epoch 149/150
493360/493360 [==============================] - 13s 27us/step - loss: 0.4519 - mean_squared_error: 0.4519 - mean_absolute_error: 0.2918
Epoch 150/150
493360/493360 [==============================] - 14s 27us/step - loss: 0.4534 - mean_squared_error: 0.4534 - mean_absolute_error: 0.2838
(493360, 25)
(211440, 25)


In [7]:
# # combine the viirs and imputed calipso data
# x_train_pt = np.concatenate((x_train_v, x_train_pt),axis=1)
# print(x_train_pt.shape)
# x_valid_pt = np.concatenate((x_valid_v, x_valid_pt),axis=1)
# print(x_valid_pt.shape)
# x_test_pt = np.concatenate((x_test_v, x_test_pt),axis=1)
# print(x_test_pt.shape)

In [8]:
# xt_train = x_train_v
# xt_valid = x_valid_v
# xt_test = x_test_v

# xs_train = x_train_pt[:, 0:np.shape(xt_train)[1]]
# xs_valid = x_valid_pt[:, 0:np.shape(xt_valid)[1]]
# xs_test= x_test_pt[:, 0:np.shape(xt_test)[1]]
# xs_train = x_train_pt[:, 5:25]
# xs_valid = x_valid_pt[:, 5:25]
# xs_test= x_test_pt[:, 5:25]

xs_train = x_train_c
xs_valid = x_valid_c
# xs_test = x_test_c

xt_train = x_train_pt 
xt_valid = x_valid_pt 
# xt_test = x_test_pt 

print(xs_train.shape)
print(xs_valid.shape)
# print(xs_test.shape)

(493360, 25)
(211440, 25)


In [9]:
ys_train = y_train
yt_train = y_train

ys_valid = y_valid
yt_valid = y_valid

batch_size = 1024
epochs = 30

#SAMPLING_ITERATIONS = 3000
n_iters = 300
ip_size = 25
n_classes = 6


In [10]:
from keras.layers import Input, Dense, Activation, BatchNormalization, PReLU, Dropout
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

def build_models(n_neurons):
    """Creates three different models, one used for source only training, two used for domain adaptation"""
    inputs = Input(shape=(n_neurons,)) 
    #n_classes = 6
    x1 = Dense(n_neurons, activation='relu')(inputs)
    #x1 = BatchNormalization()(x1)
    x2 = Dense(200, activation='relu')(x1)
    #x2 = BatchNormalization()(x2)
    x2 = Dropout(0.5)(x2)
    x3 = Dense(100, activation='relu')(x2)
    #x3 = BatchNormalization()(x3)
    x3 = Dropout(0.5)(x3)
    x4 = Dense(100, activation='relu')(x3)
    #x4 = BatchNormalization()(x4)
    #x4 = Activation("elu")(x4)  

    
    x5 = Dense(50, activation='relu', name="mo1")(x4)
    #x5 = BatchNormalization()(x5)
    source_classifier = Dense(n_classes, activation='softmax', name="mo")(x5)  
    
    
    domain_classifier = Dense(100, activation='relu', name="do4")(x4)
    domain_classifier = Dense(32, activation='relu', name="do5")(domain_classifier)
    #domain_classifier = BatchNormalization(name="do6")(domain_classifier)
    #domain_classifier = Activation("elu", name="do7")(domain_classifier)
    domain_classifier = Dropout(0.5)(domain_classifier)

    domain_classifier = Dense(2, activation='softmax', name="do")(domain_classifier)

    comb_model = Model(inputs=inputs, outputs=[source_classifier, domain_classifier])
    comb_model.compile(optimizer="Adam",
              loss={'mo': 'categorical_crossentropy', 'do': 'binary_crossentropy'},
              loss_weights={'mo': 2, 'do': 1}, metrics=['accuracy'], )

    source_classification_model = Model(inputs=inputs, outputs=[source_classifier])
    source_classification_model.compile(optimizer="Adam",
              loss={'mo': 'categorical_crossentropy'}, metrics=['categorical_accuracy'], )


    domain_classification_model = Model(inputs=inputs, outputs=[domain_classifier])
    domain_classification_model.compile(optimizer="Adam",
                  loss={'do': 'binary_crossentropy'}, metrics=['accuracy'])
    
    
    embeddings_model = Model(inputs=inputs, outputs=[x4])
    embeddings_model.compile(optimizer="Adam",loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])
    
    plot_model(comb_model, to_file='comb_model.png', show_shapes=True, show_layer_names=True)
    plot_model(embeddings_model, to_file='embeddings_model.png', show_shapes=True, show_layer_names=True)
    plot_model(source_classification_model, to_file='source_classifier_model.png', show_shapes=True, show_layer_names=True)
    plot_model(domain_classification_model, to_file='domain_classifier_model.png', show_shapes=True, show_layer_names=True)


    from IPython.display import Image
    Image(retina=True, filename='comb_model.png')

                        
    return comb_model, source_classification_model, domain_classification_model, embeddings_model

def batch_generator(data, batch_size):
    """Generate batches of data.

    Given a list of numpy data, it iterates over the list and returns batches of the same size
    This
    """
    all_examples_indices = len(data[0])
    while True:
        mini_batch_indices = np.random.choice(all_examples_indices, size=batch_size, replace=False)
        tbr = [k[mini_batch_indices] for k in data]
        yield tbr

In [11]:

from sklearn.preprocessing import label_binarize
from keras.utils.vis_utils import plot_model


def train(Xs, ys, Xt, yt, Xs_valid, ys_valid, Xt_valid, yt_valid, enable_dann = True, n_iterations = n_iters):
        
    model, source_classification_model, domain_classification_model, embeddings_model = build_models(ip_size)

    
    y_class_dummy = np.ones((len(Xt), 2))
    y_adversarial_1 = to_categorical(np.array(([1] * batch_size + [0] * batch_size)))
    
    sample_weights_class = np.array(([1] * batch_size + [0] * batch_size))
    sample_weights_adversarial = np.ones((batch_size * 2,))

    S_batches = batch_generator([Xs, to_categorical(ys)], batch_size)
    print (S_batches)
    T_batches = batch_generator([Xt, np.zeros(shape = (len(Xt),2))], batch_size)
    
    #print S_batches, T_batches
    
    for i in range(n_iterations):
        # # print(y_class_dummy.shape, ys.shape)
        y_adversarial_2 = to_categorical(np.array(([0] * batch_size + [1] * batch_size)))
#         print ("y_adversarial_2:", y_adversarial_2)

        X0, y0 = next(S_batches)
#         print('X0 size:', X0.shape)
#         print('X0:', X0)
        X1, y1 = next(T_batches)
#         print('X1 size:', X1.shape)
#         print('X1:', X1)

        X_adv = np.concatenate([X0, X1])
        
        #print 'X_adv shape',X_adv.shape,'X0 shape', X0.shape, 'X1 shape',X1.shape
        y_class = np.concatenate([y0, np.zeros_like(y0)])

        adv_weights = []
        for layer in model.layers:
            if (layer.name.startswith("do")):
                adv_weights.append(layer.get_weights())

        if(enable_dann):
            # note - even though we save and append weights, the batchnorms moving means and variances
            # are not saved throught this mechanism 
            
            #print 'y class shape',y_class.shape,'y_adv1  shape',y_adversarial_1.shape

            
            stats = model.train_on_batch(X_adv, [y_class, y_adversarial_1],
                                     sample_weight=[sample_weights_class, sample_weights_adversarial])
            
            k = 0
            for layer in model.layers:
                if (layer.name.startswith("do")):
                    layer.set_weights(adv_weights[k])
                    k += 1

            class_weights = []
            
        
            for layer in model.layers:
                if (not layer.name.startswith("do")):
                    class_weights.append(layer.get_weights())
            
            
            #print 'y_adv2  shape',y_adversarial_2.shape
            stats2 = domain_classification_model.train_on_batch(X_adv, [y_adversarial_2])

            k = 0
            for layer in model.layers:
                if (not layer.name.startswith("do")):
                    layer.set_weights(class_weights[k])
                    k += 1

        else:
            source_classification_model.train_on_batch(X0,y0)
            
       
        if ((i + 1) % 100 == 0):
            # print(i, stats)
            ##y_test_hat_t = source_classification_model.predict(Xt).argmax(1)
            ##y_test_hat_s = source_classification_model.predict(Xs).argmax(1)
            ##print("Iteration %d, source accuracy =  %.3f, target accuracy = %.3f"%(i, accuracy_score(ys, y_test_hat_s), accuracy_score(yt, y_test_hat_t)))
            
            # calculate train source and target accuracy
            y_test_hat_t = source_classification_model.predict(Xt).argmax(1)
            y_test_hat_s = source_classification_model.predict(Xs).argmax(1)
            
            #print y_test_hat_t.shape, yt.shape
            #print y_test_hat_t, yt.reshape(-1)
            
            y_test_hat_t1 = keras.utils.to_categorical(np.asarray(y_test_hat_t)-1, n_classes)
            y_test_hat_s1 = keras.utils.to_categorical(np.asarray(y_test_hat_s)-1, n_classes)
            yt1 = keras.utils.to_categorical(np.asarray(yt)-1, n_classes)
            ys1 = keras.utils.to_categorical(np.asarray(ys)-1, n_classes)

            print("Iteration %d, source train accuracy =  %.3f, target train accuracy = %.3f"%(i, accuracy_score(ys, y_test_hat_s), accuracy_score(yt, y_test_hat_t)))
            print("Iteration %d, source train auroc =  %.3f, target train auroc = %.3f"%(i, roc_auc_score(ys1, y_test_hat_s1), roc_auc_score(yt1, y_test_hat_t1)))
            
            # calculate valid source and target accuracy
            y_valid_hat_t = source_classification_model.predict(Xt_valid).argmax(1)
            y_valid_hat_s = source_classification_model.predict(Xs_valid).argmax(1)
            
            y_valid_hat_t1 = keras.utils.to_categorical(np.asarray(y_valid_hat_t)-1, n_classes)
            y_valid_hat_s1 = keras.utils.to_categorical(np.asarray(y_valid_hat_s)-1, n_classes)
            
            yt1_valid = keras.utils.to_categorical(np.asarray(yt_valid)-1, n_classes)
            ys1_valid = keras.utils.to_categorical(np.asarray(ys_valid)-1, n_classes)
            
            print("Iteration %d, source valid accuracy =  %.3f, target valid accuracy = %.3f"%(i, accuracy_score(ys_valid, y_valid_hat_s), accuracy_score(yt_valid, y_valid_hat_t)))
            print("Iteration %d, source valid auroc =  %.3f, target valid auroc = %.3f"%(i, roc_auc_score(ys1_valid, y_valid_hat_s1), roc_auc_score(yt1_valid, y_valid_hat_t1)))
            
    return source_classification_model

In [12]:
embs = train(xs_train, ys_train, xt_train, yt_train, xs_valid, ys_valid, xt_valid, yt_valid, enable_dann = True)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
<generator object batch_generator at 0x1a2bf3a318>
Iteration 99, source train accuracy =  0.983, target train accuracy = 0.774
Iteration 99, source train auroc =  0.985, target train auroc = 0.840
Iteration 99, source valid accuracy =  0.983, target valid accuracy = 0.766
Iteration 99, source valid auroc =  0.986, target valid auroc = 0.834
Iteration 199, source train accuracy =  0.993, target train accuracy = 0.776
Iteration 199, source train auroc =  0.994, target train auroc = 0.846
Iteration 199, source valid accuracy =  0.994, target valid accuracy = 0.767
Iteration 199, source valid auroc =  0.995, target valid auroc = 0.840
Iteration 299, source train accuracy =  0.996, target train accuracy = 0.775
Iteration 299, source train auroc =  0.997, target train auroc = 0.847
Iteration 299, source valid accuracy =  0.996, target valid accuracy = 0.767
Iteration 299, source

In [13]:
# # Final Trained model 
# y_valid_hat_t = embs.predict(xt_valid).argmax(1)
# y_valid_hat_s = embs.predict(xs_valid).argmax(1)

# y_valid_hat_t1 = keras.utils.to_categorical(np.asarray(y_valid_hat_t)-1, n_classes)
# y_valid_hat_s1 = keras.utils.to_categorical(np.asarray(y_valid_hat_s)-1, n_classes)

# yt1_valid = keras.utils.to_categorical(np.asarray(yt_valid)-1, n_classes)
# ys1_valid = keras.utils.to_categorical(np.asarray(ys_valid)-1, n_classes)

# print("Final trained model, source valid accuracy =  %.3f, target valid accuracy = %.3f"%(accuracy_score(ys_valid, y_valid_hat_s), accuracy_score(yt_valid, y_valid_hat_t)))
# print("Final trained model, source valid auroc =  %.3f, target valid auroc = %.3f"%(roc_auc_score(ys1_valid, y_valid_hat_s1), roc_auc_score(yt1_valid, y_valid_hat_t1)))


In [14]:
# data_test = np.load('/content/content/My Drive/Colab Notebooks/kddworkshop/fulldata/test_144_day.npz')
data_test = np.load('test_144_day.npz')

passive =1 

#load common data
latlon_test = data_test['latlon']
iff_test = data_test['iff']

# if passive ==1:
x_t_test = data_test['viirs']
y_t_test = data_test['label']
# else:
x_s_test = data_test['calipso']
y_s_test = data_test['label']
    
inds_test,vals_test = np.where(y_t_test>0)

# process common data
Latlon_test = latlon_test[inds_test]
Iff_test = iff_test[inds_test]

Y_t_test = y_t_test[inds_test]
X_t_test = x_t_test[inds_test]

Y_s_test = y_s_test[inds_test]
X_s_test = x_s_test[inds_test]

# 0 =< SZA <= 83
print('original X_t_test: ', X_t_test.shape)
rows_test = np.where((X_t_test[:,0] >= 0) & (X_t_test[:,0] <= 83) & (X_t_test[:,15] > 100) & (X_t_test[:,15] < 400) & (X_t_test[:,16] > 100) & (X_t_test[:,16] < 400) & (X_t_test[:,17] > 100) & (X_t_test[:,17] < 400) & (X_t_test[:,18] > 100) & (X_t_test[:,18] < 400) & (X_t_test[:,19] > 100) & (X_t_test[:,19] < 400) & (X_t_test[:,10] > 0))
print("rows_test:", rows_test)
print("rows_test.shape:", len(rows_test))

Latlon_test = Latlon_test[rows_test]
Iff_test = Iff_test[rows_test]

Y_t_test = Y_t_test[rows_test]
X_t_test = X_t_test[rows_test]

Y_s_test = Y_s_test[rows_test]
X_s_test = X_s_test[rows_test]

X_s_test = np.nan_to_num(X_s_test)
X_t_test = np.nan_to_num(X_t_test)

print('after SZA X_t_test: ', X_t_test.shape)
print('after SZA X_s_test: ', X_s_test.shape)

#concanate common data
# X_t_test = np.concatenate((X_t_test, Latlon_test, Iff_test), axis=1)
X_s_test = np.concatenate((X_s_test, Latlon_test, Iff_test), axis=1)

print (X_s_test.shape)
print (X_t_test.shape)

X_test=np.concatenate((X_t_test, X_s_test), axis=1)

x_test2=sc_X.transform(X_test)

X_t_test = x_test2[:, 0:20]
x_test_c2 = x_test2[:, 20:45]
x_test_comm2 = x_test2[:, 45:51]

original X_t_test:  (494657, 20)
rows_test: (array([     0,      1,      2, ..., 494654, 494655, 494656]),)
rows_test.shape: 1
after SZA X_t_test:  (208608, 20)
after SZA X_s_test:  (208608, 25)
(208608, 31)
(208608, 20)


In [15]:
# MLP regressor transform the viirs data
x_test_pt = model_reg.predict(X_t_test)
print(x_test_pt)

# x_test_pt_test = np.concatenate((x_test_pt, x_test_comm2),axis=1)
x_test_pt_test = x_test_pt
print(x_test_pt_test.shape)

yt_test = Y_t_test - 1

[[-0.75204325 -0.60750973 -0.36847982 ... -0.022455    1.8166993
   0.32035637]
 [-0.71309817 -0.58306277 -0.5306941  ... -0.01449026  1.8505003
   0.26749146]
 [-0.8837462  -0.7165735  -0.6131298  ... -0.00974993  1.885965
   0.23349693]
 ...
 [-0.94342333 -0.72023124 -0.75856835 ...  0.03272877  0.596092
   0.1441794 ]
 [-0.7397635  -0.5320477  -0.6472182  ...  0.0152457  -0.00934259
   0.3145504 ]
 [-1.195288   -0.92080253 -0.81429476 ...  0.03569934  0.588303
  -0.06273407]]
(208608, 25)


In [16]:
# Predict on test_xxx_day data
y_test_hat_t = embs.predict(x_test_pt_test).argmax(1)

y_test_hat_t1 = keras.utils.to_categorical(np.asarray(y_test_hat_t)-1, n_classes)

yt1_test = keras.utils.to_categorical(np.asarray(yt_test)-1, n_classes)

print("Final trained model, target test_xxx_day accuracy = %.3f"%(accuracy_score(yt_test, y_test_hat_t)))
print("Final trained model, target test_xxx_day auroc = %.3f"%(roc_auc_score(yt1_test, y_test_hat_t1)))


Final trained model, target test_xxx_day accuracy = 0.602
Final trained model, target test_xxx_day auroc = 0.739


In [ ]:
# Data analysis Below, No need to see/use below this line for modeling

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on 12-10-2019

@author: Xin Huang
"""

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop


from sklearn.model_selection import train_test_split
import numpy as np

# data = np.load('Sat_data_small.npz')
data = np.load('train10.npz')

# passive = 1

#load common data
latlon = data['latlon']
iff = data['iff']

X_v = data['viirs']
Y_v = data['label']
print ('X_v shape:');
print (X_v.shape);

X_c = data['calipso']
Y_c = data['label']
print ('X_c shape:');
print (X_c.shape);

inds,vals = np.where(Y_v>0)

# process common data
Latlon = latlon[inds]
Iff = iff[inds]

Y_v = Y_v[inds]
X_v = X_v[inds]
print ('X_v')
print (X_v)

# inds_c,vals_c = np.where(Y_c>0)

Y_c = Y_c[inds]
X_c = X_c[inds]
print ('X_c')
print (X_c)



# 0 =< SZA <= 83
print('original X_v: ', X_v.shape)
rows = np.where((X_v[:,0] >= 0) & (X_v[:,0] <= 83) & (X_v[:,15] > 100) & (X_v[:,15] < 400) & (X_v[:,16] > 100) & (X_v[:,16] < 400) & (X_v[:,17] > 100) & (X_v[:,17] < 400) & (X_v[:,18] > 100) & (X_v[:,18] < 400) & (X_v[:,19] > 100) & (X_v[:,19] < 400) & (X_v[:,10] > 0))
print("rows:", rows)
print("rows.shape:", len(rows))

Latlon = Latlon[rows]
Iff = Iff[rows]

Y_v = Y_v[rows]
X_v = X_v[rows]

Y_c = Y_c[rows]
X_c = X_c[rows]

X_c = np.nan_to_num(X_c)
X_v = np.nan_to_num(X_v)

print('after SZA X_v: ', X_v.shape)
print('after SZA X_c: ', X_c.shape)

# pca = decomposition.PCA(n_components=20)
# pca.fit(X_s)
# X_s = pca.transform(X_s)
# print (X_s.shape)

#concanate common data
# X_v = np.concatenate((X_v, Latlon, Iff), axis=1)
X_c = np.concatenate((X_c, Latlon, Iff), axis=1)

print (X_v.shape)
print (X_c.shape)

import pandas as pd 
import seaborn as sns

df = pd.DataFrame(X_v, columns=['band1', 'band2', 'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10','band11', 'band12', 'band13', 'band14', 'band15', 'band16', 'band17', 'band18','band19', 'band20'])
df['label'] = Y_v
print(df.head(5))
fig=plt.figure()
df.describe()

In [ ]:
df['label'].value_counts()

In [109]:
print (x_train_pt)


[[-0.53182329 -0.53182332 -0.53182079 ...  0.0420989   0.08963409
   0.31619685]
 [-0.53182329 -0.53182332 -0.53182079 ...  0.00996393 -0.02501277
   0.46844565]
 [ 0.01953407  0.01953405  0.0195366  ...  0.26004047  0.07456922
   0.33871202]
 ...
 [-0.53182329 -0.53182332 -0.53182079 ...  0.36861761  0.07131892
   0.23990399]
 [ 0.01953941  0.01953779  0.01953991 ...  0.15939287  0.0604022
   0.59214814]
 [ 3.63269252  3.63269249  3.63269522 ... -0.11353965 -0.03907268
  -0.54627213]]


In [110]:
from sklearn.preprocessing import StandardScaler
sc_X2 = StandardScaler()
sc_X2.fit(x_train_pt)
x_train_pt=sc_X2.transform(x_train_pt)
x_valid_pt=sc_X2.transform(x_valid_pt)
x_test_pt=sc_X2.transform(x_test_pt)

In [111]:
# use Logistic regression model
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial').fit(x_train_pt, y_train)
y_pred = clf.predict(x_test_pt)
print (clf.score(x_test_pt, y_test))

num_classes1 = 6
y_lr_gd = keras.utils.to_categorical(y_test-1, num_classes1)
y_lr_pred = keras.utils.to_categorical(y_pred-1, num_classes1)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_lr_gd, y_lr_pred)

/Users/xin/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.746409663926994


/Users/xin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8284767921594982

In [112]:
# use deep learning model
num_classes = 6

print ('y_train')
print (y_train)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train-1, num_classes)
y_valid = keras.utils.to_categorical(y_valid-1, num_classes)
y_test = keras.utils.to_categorical(y_test-1, num_classes)

print ('y_train converted')
print (y_train)

batch_size = 256
epochs = 30

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train_pt.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

history = model.fit(x_train_pt, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valid_pt, y_valid))
score = model.evaluate(x_test_pt, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test score:', score)

y_train
[[6]
 [6]
 [2]
 ...
 [2]
 [2]
 [6]]
y_train converted
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_109 (Dense)            (None, 512)               23552     
_________________________________________________________________
dropout_46 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_110 (Dense)            (None, 256)               131328    
_________________________________________________________________
dropout_47 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_111 (Dense)            (None, 128)               32896     
_________________________________________________________________
dropout_48 (Dr

In [113]:
score = model.evaluate(x_test_pt, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test score:', score)
predict_result = model.predict_proba(x_test_pt);
print (predict_result)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predict_result)

Test loss: 0.6620687058554082
Test accuracy: 0.7639786676032213
Test score: [0.6620687058554082, 0.7639786676032213]
[[1.46878022e-03 5.26535511e-01 5.38771898e-02 1.71978876e-01
  7.05717830e-03 2.39082426e-01]
 [5.75361044e-11 2.02527426e-05 9.79642272e-01 2.04527215e-03
  4.91435266e-08 1.82922501e-02]
 [1.77109632e-06 1.12161669e-03 1.46082355e-08 2.24123795e-08
  9.33094084e-01 6.57825097e-02]
 ...
 [2.00776439e-02 1.32450342e-01 4.94697243e-01 9.42886472e-02
  3.79949361e-02 2.20491171e-01]
 [8.04204121e-02 7.15925358e-03 4.13641427e-03 1.36366025e-05
  8.28366876e-01 7.99033195e-02]
 [2.30168662e-05 4.75512724e-03 3.51433840e-07 6.42637588e-07
  9.00022626e-01 9.51981917e-02]]


0.9489682439851773

In [71]:
# Experiments, references 

# use deep learning model
batch_size = 256
epochs = 15

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valid, y_valid))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test score:', score)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 512)               23552     
_________________________________________________________________
dropout_31 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_74 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_32 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_75 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_33 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_76 (Dense)             (None, 64)                8256      
__________

In [72]:
predict_result = model.predict_proba(x_test);
print (predict_result)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predict_result)

[[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]]


0.9996603756408294

In [17]:
# references

from sklearn.linear_model import LinearRegression
mlr = LinearRegression()
# Fit linear regression.
mlr.fit(X_v, X_c[:,0])
# Get the slope and intercept of the line best fit.
print(mlr.intercept_)
print(mlr.coef_)

-0.6611631949910323
[ 3.63297909e+00 -1.01091895e+00  4.17880312e-02  1.54816115e-01
  1.61379276e+00 -6.09930324e-01 -3.79627864e+00  1.35149422e-04
  1.26073034e-03  2.61727207e-01  1.29948456e-02  2.37170662e-01
 -1.33159116e-01  3.56930404e-03 -3.91941027e-01 -5.58686480e-04
 -1.45929821e-04 -3.79621786e-05 -1.05407018e-02 -3.11334816e-03]


In [10]:
X=np.concatenate((X_v, X_c), axis=1)
Y=Y_v
print (X.shape)
print (Y_v)

(341889, 45)
[[3]
 [3]
 [3]
 ...
 [6]
 [6]
 [6]]


In [11]:
x_train, x_temp, y_train, y_temp = train_test_split(X, Y,
                                                    test_size=0.6,
                                                    random_state=0,
                                                    stratify=Y)

x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp,
                                                    test_size=0.5,
                                                    random_state=0,
                                                    stratify=y_temp)

In [18]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()  
regressor.fit(x_train, y_train)
print(regressor.intercept_)
print(regressor.coef_)

[-0.32186763  0.06481409 -0.55881887 -0.07118032  0.2677794   1.61927334]
[[-2.68158752e+05  1.36456457e+05  2.34556886e+04  8.17350437e+04
  -3.26284936e+05  6.10197570e+05 -1.86717785e+05  1.15136665e-03
  -4.23970582e-02  4.61985570e+04  3.93571740e+00 -8.99035966e+04
  -3.51468101e+04  3.21619644e+00  8.16656542e+03 -5.64750382e+00
  -2.94516298e-03  3.27020060e-02  7.04622481e+00 -1.06227746e+01
   7.77573029e-02 -7.82765662e-02 -4.98097120e-01  1.73382745e-01
  -9.23511099e-02  1.41456985e-01  2.23518550e+00 -2.62107860e-01
  -1.93921942e+00  4.74158021e-02 -3.01991512e-01  2.65116590e-01
  -1.66443332e-02  6.09429334e-02 -8.28398469e-02 -2.04355877e-01
   5.08505685e-02  4.22842343e-01  1.43619552e-01  8.35576448e-02
  -1.69183285e-01 -1.42307477e-02 -8.89169534e-02  1.27865461e-01
  -2.23144599e-02]
 [ 2.11309529e+05 -5.96722870e+04  8.92815771e+03 -5.18681658e+04
  -1.35859512e+03  1.05206316e+03 -1.07246505e+05 -8.95203165e-03
   2.50560537e-02 -4.82359736e+04 -3.52553311e+00

In [12]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial').fit(x_train, y_train)
y_pred = clf.predict(x_test)
print (clf.score(x_test, y_test))

/Users/xin/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9979818070139519


/Users/xin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [13]:
num_classes1 = 6
y_lr_gd = keras.utils.to_categorical(y_test-1, num_classes1)
y_lr_pred = keras.utils.to_categorical(y_pred-1, num_classes1)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_lr_gd, y_lr_pred)


0.9984782962250461

In [14]:
num_classes = 6

print ('y_train')
print (y_train)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train-1, num_classes)
y_valid = keras.utils.to_categorical(y_valid-1, num_classes)
y_test = keras.utils.to_categorical(y_test-1, num_classes)

print ('y_train converted')
print (y_train)

y_train
[[6]
 [6]
 [2]
 ...
 [2]
 [2]
 [6]]
y_train converted
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [15]:
batch_size = 256
epochs = 30

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valid, y_valid))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test score:', score)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               23552     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_3 (Dropout)  

In [16]:
predict_result = model.predict_proba(x_test);
print (predict_result)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predict_result)

[[0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00
  1.1416998e-31]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00
  0.0000000e+00]
 ...
 [0.0000000e+00 7.6100818e-08 1.8864904e-18 2.7485150e-26 0.0000000e+00
  9.9999988e-01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00
  0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00
  0.0000000e+00]]


0.9997734130200948

In [8]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
from dask.distributed import progress

cluster = SLURMCluster(cores=1, memory='4 GB', job_extra=['--exclusive','--qos=normal','--time=03:00:00'])
cluster.scale(5)

client = Client(cluster)
print(cluster.job_script())

/Users/xin/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/xin/anaconda3/lib/python3.7/site-packages/dask_jobqueue/config.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/Users/xin/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
tornado.application - ERROR - Exception in callback functools.partial(<bound method JobQueueCluster.scale_up of SLURMCluster(cores=0, memory=0 B, workers=0/0, jobs=0/0)>, 5)
Traceback (most recent call last

#!/bin/bash

#!/usr/bin/env bash
#SBATCH -J dask-worker
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=4G
#SBATCH -t 00:30:00
#SBATCH --exclusive
#SBATCH --qos=normal
#SBATCH --time=03:00:00
JOB_ID=${SLURM_JOB_ID%;*}



/Users/xin/anaconda3/bin/python -m distributed.cli.dask_worker tcp://192.168.1.166:57696 --nthreads 1 --memory-limit 4.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60

